# K Top Synonyms
by Collin Shaffer

## Introduction

My original plan for this project was indeed rather vague, which eventually became my achilles heel. In order to have a program "understand," it must be able to parse sentences, apply parts-of-speech to each word, determine word senses, and much more. What I was hoping to achieve was apparently the holy grail of a field where even the amateurs seem to have master's degrees. I did realize, only too late, that as I was writing code to anaylze the dictionary and thesaurus I had found, I was actually getting closer to a model for Word Sense Disambiguation. If I had used a corpus of sentences rather than a dictionary, perharps the results could have been more conclusive.

Essentially, what I wrote a method that turns a list of words into a larger list of words (that are synonyms of the first list), takes the unique words amongst them, removes any that aren't in the list of defined roots, and returns the top $k$ of them.

## I. Methods

### A. The Data

As originally proposed, the data for the dictionary used is a taken from [this dataset](https://github.com/adambom/dictionary)  found on GitHub, uploaded by user `adambom`. The `dictionary.json` file contains a dictionary, in the form of a dictionary! (That is, of the format {"word":"definition"}.) The thesaurus data was taken from the [Moby Poroject](http://icon.shef.ac.uk/Moby/), an early 1990's natural language processing project by Grady Ward, now made available for free by [Project Gutenberg](https://www.gutenberg.org/files/3202/).

The dictionary was read in using the eval function to keep the data as a dictionary. The thesaurus data was read in as a list via the csv package. Both the dictionary and the thesausrus were then individually separated into two sub-dictionaries based on keys that are either single words or phrases. 

The dictionary contains 83,684 defined single-word entries and 2,352 defined phrases, making a total of 86,036 definitions. The thesaurus contains 25,337 root words and 4,922 root phrases, totaling in 30,259. For simplicity, we will only be dealing with the single-word entries.

Of the 83,684 single-word entries, 14,999, about 17.92%, are thesaurus roots. Of 1,157,056 total words used in definitions, 145,077 (12.54%) are unique and, of those, 17,403 (12.00%) are thesaurus roots. This means 17403/25337, or 68.69%, thesaurus roots are used in definitions.

 Of the single-word roots, 2,758,284 words are listed as synonyms with just 69,857 (2.53%) are unique. Yet 25,222 of those unique synonyms are also roots, meaning roughly 99.55% thesaurus roots are also synonyms for other roots.

In [3]:
import csv
import numpy as np

In [5]:
def getDictionaryAndThesaurus():
    '''
    Reads in the dictionary and the thesaurus, separates out the phrases
    Returns two dictionaries {entry:definition} and two thesauruses {root:synonyms}, where for each, the keys are single words and multiple words respectively.
    '''
# def makeDictionaryData():
    with open('dictionary.json') as dictFile:
        dictionary = eval(dictFile.read())
    #since dictionary is already a {}, first make pDictionary, then remove phrases from dictionary
    pDictionary = {}
    for k,v in dictionary.items():
        if ' ' in k:
            pDictionary[k] = v
    for k in pDictionary.keys():
        if k in dictionary.keys():
            dictionary.pop(k)

# def makeThesaurusData():
    # simple fix - how to read in a csv with different length lines - http://www.gossamer-threads.com/lists/python/python/1151633
    with open('mthesaur.txt') as file:
        thesaurusData = list(csv.reader(file))
        
    # separate the phrase thesaurus & make both dictionaries
    thesaurus = {}
    pThesaurus = {}
    for i in range(len(thesaurusData)):
        ## assign sets 
        if ' ' in thesaurusData[i][0]:
            pThesaurus[thesaurusData[i][0]] = thesaurusData[i][1:]
        else:
            thesaurus[thesaurusData[i][0]] = thesaurusData[i][1:]
  
    return dictionary, pDictionary, thesaurus, pThesaurus

In [6]:
def getUniqueCharacters(dct):#, pDct, ths, pThs):
    '''
        This determined how many unique ascii characters are used in the definitions.
        Necessary for minimizing the length of the character frequency array, as the alternate
        length to generically cover the entire UTF-8 ascii character set would be 1,114,112!
    '''
    totalUniqueChars = set(())
    entries = list(dct.keys())
    defs = list(dct.values())
#     pEntries= list(pDct.keys())
#     pDefs = list(pDct.values())
#Single-word
    # defined singe words
    for i in range(len(entries)):
        temp = []
        temp.extend(entries[i]) # turn string into char array
        for c in temp:
            totalUniqueChars.add(c)
    # singe word defs
    for i in range(len(defs)):
        temp = []
        temp.extend(defs[i]) # turn string into char array
        for c in temp:
            totalUniqueChars.add(c)

#Phrase
    # defined phrases
#     for i in range(len(pEntries)):
#         temp = []
#         temp.extend(pEntries[i]) # turn string into char array
#         for c in temp:
#             totalUniqueChars.add(c)
#     # phrase definitions   
#     for i in range(len(pDefs)):
#         temp = []
#         temp.extend(pDefs[i]) # turn string into char array
#         for c in temp:
#             totalUniqueChars.add(c)

    #print(len(totalUniqueChars))

    nonalphanumerics = set(())
    for c in totalUniqueChars:
        if c != ' ' and c < '0' or (c > '9' and c < 'A') or (c > 'Z' and c < 'a') or c > 'z': #also not space
            nonalphanumerics.add(c)

    uchrs = list(totalUniqueChars)
    uchrs.sort()
    print('{} unique characters and {} nonalphanumeric characters (that aren\'t space)'.format(len(uchrs), len(nonalphanumerics)))
    return uchrs, nonalphanumerics

In [7]:
dictionary, pDictionary, thesaurus, pThesaurus = getDictionaryAndThesaurus()
uniqueCharacters, uniqueNonAlphaNumerics = getUniqueCharacters(dictionary)#, pDictionary, thesaurus, pThesaurus)
len(dictionary), len(pDictionary), len(thesaurus), len(pThesaurus)

129 unique characters and 66 nonalphanumeric characters (that aren't space)


(83684, 2352, 25337, 4922)

In [8]:
entryWrdsWithSnnms = set(())
for entry in dictionary.keys():
    for char in uniqueNonAlphaNumerics:
        entry = entry.strip(char)
    if entry.lower() in thesaurus.keys():
        entryWrdsWithSnnms.add(entry)
        
print('{} out of {} ({:.2f}%) dictionary entries have synonyms.'
      .format(len(entryWrdsWithSnnms), len(dictionary.keys()), (len(entryWrdsWithSnnms)/len(dictionary.keys()))*100))

14999 out of 83684 (17.92%) dictionary entries have synonyms.


In [9]:
totalDefWrds = 0
defWrdsWithSnnms = {}# essentially expand each dw into a dictionary of snnms
theSetOfUniqueDefWords = set(())
for defi in dictionary.values():#defi is a string
    for wrdj in defi.split():#wrdj is the list of words in defi
        for char in uniqueNonAlphaNumerics:
            wrdj = wrdj.strip(char)
        totalDefWrds += 1
        if wrdj not in theSetOfUniqueDefWords:
            theSetOfUniqueDefWords.add(wrdj)
            if wrdj in thesaurus.keys():
                defWrdsWithSnnms[wrdj] = thesaurus[wrdj]

print('Out of {} total words used in definitions, {} ({:.2f}%) are unique.\nOf those {}, {} ({:.2f}%) are thesaurus roots.'
      .format(totalDefWrds, len(theSetOfUniqueDefWords), (len(theSetOfUniqueDefWords)/totalDefWrds)*100, 
              len(theSetOfUniqueDefWords), len(defWrdsWithSnnms), (len(defWrdsWithSnnms)/len(theSetOfUniqueDefWords))*100))
print('This means {}/{}, or {:.2f}%, thesaurus roots are used in definitions.'
      .format(len(defWrdsWithSnnms), len(thesaurus.keys()), (len(defWrdsWithSnnms)/len(thesaurus.keys()))*100))

Out of 1157051 total words used in definitions, 144862 (12.52%) are unique.
Of those 144862, 17423 (12.03%) are thesaurus roots.
This means 17423/25337, or 68.77%, thesaurus roots are used in definitions.


In [10]:
theSetOfUniqueSnnms = set(())
snnmsWithSnnms = set(())

totalSnnms = 0

for i in thesaurus.values():
    for j in i:
        if ' ' in j:
            for k in j.split():
                totalSnnms += 1
                if k not in theSetOfUniqueSnnms:
                    theSetOfUniqueSnnms.add(k)
                    if k in thesaurus.keys():
                        snnmsWithSnnms.add(k)
        else:
            totalSnnms += 1
            if j not in theSetOfUniqueSnnms:
                theSetOfUniqueSnnms.add(j)
                if j in thesaurus.keys():
                    snnmsWithSnnms.add(j)

print('Out of {} words listed as synonyms, {} ({:.2f}%) are unique.\nOf those {}, {} ({:.2f}%)are also roots.'
      .format(totalSnnms, len(theSetOfUniqueSnnms), (len(theSetOfUniqueSnnms)/totalSnnms)*100, 
              len(theSetOfUniqueSnnms), len(snnmsWithSnnms), (len(snnmsWithSnnms)/len(theSetOfUniqueSnnms))*100))
print('This means {}/{}, or {:.2f}%, thesaurus roots are also synonyms.'
      .format(len(snnmsWithSnnms), len(thesaurus.keys()), (len(snnmsWithSnnms)/len(thesaurus.keys()))*100))

Out of 2758284 words listed as synonyms, 69857 (2.53%) are unique.
Of those 69857, 25222 (36.11%)are also roots.
This means 25222/25337, or 99.55%, thesaurus roots are also synonyms.


### B. Machine Learning Algoriths

#### K Top Synonym

The idea here is, instead of using the euclidean distance to determine "nearest" neighbors as in KNN, each definition is first turned into a list of synonyms, then highest matching number of shared synonyms determines the class.

In [11]:
'''
Based on the KNN class from lecture notebook 28
'''
######################################################################
### class K Top Synonym
######################################################################

class KTS(object):
    
    def __init__(self, allLevels=False):
        self.X = None  # data will be stored here
        self.T = None  # class labels will be stored here
        self.dictionary = {}
        self.thesaurus = {}
#         self.XdefWrdRoots = {}# essentially expand each dw into a dictionary of snnms
        
    def getRootsPerDef(self, X):
        theSetOfUniqueDefWords = set(())
        allRoots = []
        for defi in X:#defi is a string
            defRoots = []
            for wrdj in defi.split():#wrdj is the list of words in defi
                for char in uniqueNonAlphaNumerics:
                    wrdj = wrdj.strip(char)
                if wrdj not in theSetOfUniqueDefWords:
                    theSetOfUniqueDefWords.add(wrdj)
                    if wrdj in self.thesaurus.keys():
                        defRoots += self.thesaurus[wrdj]
            allRoots.append(defRoots)
        return allRoots
    
    def train(self,X,T, dictionary,thesaurus):
        self.X = X
        self.T = T
        self.dictionary = dictionary
        self.thesaurus = thesaurus
#         self.XdefWrdsRoots = self.getRoots(self.X)
        
    def use(self, Xnew, k=1):
        Xroots = self.getRootsPerDef(Xnew)
        classes = []
        for i in range(len(Xroots)):
            uniqueDefRootsTest = [(w,int(f)) for w,f in np.array(np.unique(Xroots[i], return_counts=True) ).T]

            uniqueDefRootsThatMacthTRoots = []
            for i in uniqueDefRootsTest:
                if i[0] in T:
                    uniqueDefRootsThatMacthTRoots.append(i)

            classesForThisRoot = []
            for i in range(k):
                maxFreqTst = ('',0)
                for j in uniqueDefRootsThatMacthTRoots:
                    if j[1] > maxFreqTst[1]:
                        maxFreqTst = j
                classesForThisRoot.append(maxFreqTst[0])
                if maxFreqTst in uniqueDefRootsThatMacthTRoots:
                    uniqueDefRootsThatMacthTRoots.remove(maxFreqTst)
            classes.append(classesForThisRoot)
        return classes

### C. Experiments

X will be the definitions for words that have thesaurus roots, and T will be those thesaurus roots. Xnew will be the remaining, or a subset of, definitions that will be classified based on the synonyms of the words in their definitions.

In [12]:
# i should be in dictionary.keys(), that was the condition for items being added to entryWrdsWithSnnms, 
# but this doesn't work without rechecking for some reason...
X = []
T = []
for i in entryWrdsWithSnnms:
    if i in dictionary.keys():
        X.append(dictionary[i])
    T.append(i.lower())
Xnew = [dictionary[w] for w in set(dictionary.keys()).difference(entryWrdsWithSnnms)]
len(X), len(T), len(Xnew), len(entryWrdsWithSnnms), len(Xnew)+len(entryWrdsWithSnnms)

(14985, 14999, 68699, 14999, 83698)

In [13]:
kts = KTS()
kts.train(X,T,dictionary,thesaurus)

In [ ]:
classes = kts.use(Xnew)

In [12]:
classes[444], Xnew[444]

(['order'],
 'A tribute by the head; a capitation tax. [Written also chevageand chivage.] [Obs.]')

## II. Results

In terms of evaluating the results, since Xnew was actually chosen from words that <i>don't</i> have thesaurus roots, without partitioning X and T for Xnew so that we could actually compare accuracy, correctness is really just a matter of opinion. In short, what we have here are a couple of lists that let us determine how close the relationship is between a words and the most frequently occurring synonyms of the words in its definition from Webster's Unabridged English Dictionary. Personally think 'alarming' could be a synonym of 'awesomness', but without finding suplimental input, we may never know.

In [13]:
offset = 44
for i in range(20):
    for k,v in dictionary.items():
        if v == Xnew[i+offset]:
            print('{:10}\t{}'.format(k,classes[i+offset]))

CHLOROPHYLL	['ground']
FUCHSIA   	['cast']
OATCAKE   	['cereal']
DISARMED  	['grip']
SURDINY   	['']
ALLOGENEOUS	['air']
AMNIOTA   	['part']
EWT       	['']
REREWARD  	['train']
CRESSY    	['achromic']
DISENCHAINED	['abnegation']
MIASMATIST	['article']
FEAZINGS  	['line']
DORRHAWK  	['']
-OUS      	['essential']
UT        	['crack']
TWELFTHTIDE	['aeon']
QUATRE    	['jack']
CHROMULE  	['abstract']
GREGORIAN 	['measure']


## III. Conclusion

Other possible applications that I didn't have time to explore were algorithms that followed the ideas behind support vector machines and clustering. A language-SVM could use a similar model to above, supplimented with antonyms, and/or going another level into the synonyms. If I had used a corpus of sentences rather than a dictionary, perharps the this method could have been used to determine words sense. For clustering, there is a resource called the Roget's Thesaurus, which groups synonyms together under meaning headings. This could have been very useful for the model herein, however, the only electronic Roget's Thesaures were very old, having many mistakes, not utf-8 encoded, and were not in an easily accessable form.

While I may have been on the right path for a completing a small portion of the modern model for natural language processing, I fell short by underestimating how large even this first step would be.  However, I look forward to further developing this idea to determine if the same results could ever be achieved from a simple dictionary, e.g. maybe a kernal could be made for a natural language for fast interpretation.